In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE84046"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE84046"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE84046.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE84046.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE84046.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this study includes gene expression analysis
# in human adipose tissue, so gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For the trait (Obesity):
# From the sample characteristics, we can see BMI values at row 6, which can be used to determine obesity
trait_row = 6  # screening bmi (kg/m2) values

# For gender:
# Gender information is available at row 4 (sexe: Male/Female)
gender_row = 4

# For age:
# Age is not directly provided, but birth dates are available at row 5
# We could calculate age from birth dates
age_row = 5

# 2.2 Data Type Conversion Functions

# Obesity conversion function (using BMI >= 30 as obese)
def convert_trait(value):
    try:
        if ':' in value:
            bmi_value = float(value.split(': ')[1])
            # BMI ≥ 30 is considered obese according to standard definition
            return 1 if bmi_value >= 30.0 else 0
        else:
            return None
    except (ValueError, IndexError):
        return None

# Age conversion function
def convert_age(value):
    try:
        if ':' in value:
            birth_date_str = value.split(': ')[1]
            # Extract year from birth date
            birth_year = int(birth_date_str.split('-')[0])
            # Since the study was likely conducted around 2017 (based on GEO submission date pattern),
            # we'll use this as reference year for age calculation
            study_year = 2017  # Approximate study year
            age = study_year - birth_year
            return age
        else:
            return None
    except (ValueError, IndexError):
        return None

# Gender conversion function
def convert_gender(value):
    try:
        if ':' in value:
            gender = value.split(': ')[1].strip().lower()
            if gender == 'female':
                return 0
            elif gender == 'male':
                return 1
            else:
                return None
        else:
            return None
    except (ValueError, IndexError):
        return None

# 3. Save Metadata
# Conduct initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Make directory for output file if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Extract clinical features using the clinical_data from previous steps
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use the existing clinical_data from previous steps
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Clinical data preview:", preview)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error processing clinical data: {str(e)}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers
# These identifiers appear to be microarray probe IDs (numerical IDs) rather than 
# human gene symbols which would typically be alphanumeric (like BRCA1, TP53, etc.)
# Standard human gene symbols follow HGNC nomenclature which does not use purely numeric IDs
# These are likely Illumina or Affymetrix microarray probe IDs that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# After analyzing the gene annotation preview, I need to determine which columns 
# contain the gene identifier (probe ID) and gene symbols

# Looking at the data:
# - The 'ID' column in gene_annotation matches the index in gene_data (probe IDs)
# - The 'gene_assignment' column contains gene symbol information we need to extract

# 1. Create a mapping dataframe using the appropriate columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 2. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 3. Normalize gene symbols to handle synonyms and aggregate by official symbols
gene_data = normalize_gene_symbols_in_index(gene_data)

# 4. Preview the resulting gene expression data
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First 5 genes:")
print(gene_data.index[:5])

# 5. Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data - this was already done in Step 6
# We don't need to redo this normalization
print(f"Already normalized gene data with shape: {gene_data.shape}")

# 2. Load the previously saved clinical data
clinical_data_df = pd.read_csv(out_clinical_data_file)

# Print clinical data to understand its structure
print("Clinical data preview:")
print(clinical_data_df.head())

# Set the first column as index if it's not already 
if not clinical_data_df.index.name:
    clinical_data_df.set_index(clinical_data_df.columns[0], inplace=True)

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_df, gene_data)
print(f"Shape of linked data before missing value handling: {linked_data.shape}")

# Print the first few column names to understand what's available
print("First columns in linked data:", list(linked_data.columns)[:10])

# 3. Handle missing values systematically
# The trait name from the original processing is stored in the columns
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after missing value handling: {linked_data.shape}")

# 5. Check if the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
note = "Dataset from protein quantity in energy restriction study with BMI data used for obesity assessment."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to quality issues.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the function from tools.preprocess
# Use the parameters defined in STEP 2 for the trait, age, and gender rows
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the extracted clinical features to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data before missing value handling: {linked_data.shape}")

# 4. Handle missing values systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after missing value handling: {linked_data.shape}")

# 5. Check if the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
note = "Dataset from NAFLD study with hypertension trait information available."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to quality issues.")